In [3]:
pip install tensorflow librosa numpy


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

In [5]:
# Function to extract Mel spectrograms from audio files
def extract_features(file_path, max_length=216):  # Define a max_length suitable for your dataset
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Pad or truncate MFCCs
    if mfccs.shape[1] < max_length:
        pad_width = max_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_length]

    # Pad or truncate Mel spectrogram
    if mel_db.shape[1] < max_length:
        pad_width = max_length - mel_db.shape[1]
        mel_db = np.pad(mel_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_db = mel_db[:, :max_length]

    return mfccs, mel_db

def plot_features(mfccs, mel_db, title):
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')

    plt.subplot(1, 2, 2)
    librosa.display.specshow(mel_db, x_axis='time', y_axis='mel', sr=sample_rate)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()




In [6]:
base_path = '/content/drive/MyDrive/Colab Notebooks/Birds_Sounds'
classes = ['Brown_Tinamou', 'Cinereous_Tinamou', 'Great_Tinamou']

data = []
labels = []

for label in classes:
    folder_path = os.path.join(base_path, label)
    for file in os.listdir(folder_path):
        if file.endswith('.mp3'):
            file_path = os.path.join(folder_path, file)
            mfccs, mel_db = extract_features(file_path)
            data.append((mfccs, mel_db))
            labels.append(label)

In [23]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Convert data to numpy arrays
mfccs_data = np.array([x[0] for x in data])
mel_data = np.array([x[1] for x in data])

# Split the dataset
X_train_mfcc, X_test_mfcc, y_train, y_test = train_test_split(mfccs_data, y, test_size=0.2, random_state=42)
X_train_mel, X_test_mel, _, _ = train_test_split(mel_data, y, test_size=0.2, random_state=42)


In [24]:
def build_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    return model

input_shape = (mfccs_data.shape[1], mfccs_data.shape[2], 1)  # Adjust shape as needed

model = build_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

X_train_mfcc = X_train_mfcc[..., np.newaxis]  # Add channel dimension
X_test_mfcc = X_test_mfcc[..., np.newaxis]

history = model.fit(X_train_mfcc, y_train, epochs=20, validation_data=(X_test_mfcc, y_test))


Epoch 1/20
4/4 [==============================] - 2s 73ms/step - loss: 30.2899 - accuracy: 0.3652 - val_loss: 6.6001 - val_accuracy: 0.3103
Epoch 2/20
4/4 [==============================] - 0s 19ms/step - loss: 2.8694 - accuracy: 0.3478 - val_loss: 1.0936 - val_accuracy: 0.2759
Epoch 3/20
4/4 [==============================] - 0s 19ms/step - loss: 1.0534 - accuracy: 0.4522 - val_loss: 1.0580 - val_accuracy: 0.4483
Epoch 4/20
4/4 [==============================] - 0s 19ms/step - loss: 1.0243 - accuracy: 0.5043 - val_loss: 1.2871 - val_accuracy: 0.4828
Epoch 5/20
4/4 [==============================] - 0s 19ms/step - loss: 0.8995 - accuracy: 0.6609 - val_loss: 0.9291 - val_accuracy: 0.5862
Epoch 6/20
4/4 [==============================] - 0s 19ms/step - loss: 0.8324 - accuracy: 0.6087 - val_loss: 0.9638 - val_accuracy: 0.4828
Epoch 7/20
4/4 [==============================] - 0s 21ms/step - loss: 0.6742 - accuracy: 0.7826 - val_loss: 0.8032 - val_accuracy: 0.5862
Epoch 8/20
4/4 [==========

In [25]:
test_loss, test_acc = model.evaluate(X_test_mfcc, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.2f}')


1/1 - 0s - loss: 0.4850 - accuracy: 0.8276 - 22ms/epoch - 22ms/step
Test accuracy: 0.83


In [15]:
train_loss, train_acc = model.evaluate(X_train_mfcc, y_train, verbose=2)
print(f'Train accuracy: {train_acc:.2f}')




4/4 - 0s - loss: 2.5096e-04 - accuracy: 1.0000 - 48ms/epoch - 12ms/step
Train accuracy: 1.00


In [11]:
def predict_bird_species(file_path):
    mfccs, _ = extract_features(file_path)
    mfccs = mfccs[..., np.newaxis]
    mfccs = np.expand_dims(mfccs, axis=0)  # Add batch dimension
    predictions = model.predict(mfccs)
    predicted_label = np.argmax(predictions, axis=1)
    return label_encoder.inverse_transform(predicted_label)[0]

# Example usage
file_path = '/content/drive/MyDrive/Colab Notebooks/Birds_Sounds/Cinereous_Tinamou/XC387680 - Cinereous Tinamou - Crypturellus cinereus.mp3'
print(predict_bird_species(file_path))

1/1 [==============================] - 1s 527ms/step
Cinereous_Tinamou


In [12]:
model.save('bird_sound_classification_model2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
